In [1]:
from app.database import SessionLocal
from sqlalchemy.orm import aliased
from sqlalchemy import func, text
from app import models
import time

In [3]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings


In [5]:
client = chromadb.PersistentClient(path="./dbmovies_default",settings=Settings(allow_reset=True))

In [28]:
collection = client.get_or_create_collection(name="movies")

In [26]:
query = text("""
   SELECT
    m.movieId,
    m.title,
    m.genres,
    ROUND(AVG(r.rating), 2) AS average_rating,
    COALESCE(
        (
            SELECT REPLACE(GROUP_CONCAT(DISTINCT t.tag), ',', ' ')
            FROM tags t
            WHERE m.movieId = t.movieId
        ),
        ''
    ) AS tags
FROM
    movies m
LEFT JOIN
    ratings r ON m.movieId = r.movieId
GROUP BY
    m.movieId, m.title, m.genres;
""")

In [29]:
client.list_collections()

[Collection(name=movies)]

In [27]:
client.delete_collection(name="movies")
client.list_collections()

[]

In [30]:
#insertar por chinks 

with SessionLocal() as db:
    results = db.execute(query).fetchall()

    # Imprimir los resultados
    for result in results:
        movieId,title, genres, average_rating, tags = result
        tags_list = tags.split(',') if tags else []
        tags_str = ' '.join(tags_list)
        movie_info = f"{title} {genres} {average_rating} {tags_str}"
        movie_metada = title
        movie_id = movieId
        #RMP= 3
        #TOTAL = 9742
        
        collection.add(
            documents=movie_info,
            metadatas={"title":movie_metada},
            ids=str(movieId)
        )
        print(f"{movie_id} {title} {genres} {average_rating} {tags_str}")
        # time.sleep(20)
        

1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy 3.92 pixar fun
2 Jumanji (1995) Adventure|Children|Fantasy 3.43 Robin Williams fantasy magic board game game
3 Grumpier Old Men (1995) Comedy|Romance 3.26 moldy old
4 Waiting to Exhale (1995) Comedy|Drama|Romance 2.36 
5 Father of the Bride Part II (1995) Comedy 3.07 pregnancy remake
6 Heat (1995) Action|Crime|Thriller 3.95 
7 Sabrina (1995) Comedy|Romance 3.19 remake
8 Tom and Huck (1995) Adventure|Children 2.88 
9 Sudden Death (1995) Action 3.13 
10 GoldenEye (1995) Action|Adventure|Thriller 3.5 
11 American President, The (1995) Comedy|Drama|Romance 3.67 politics president
12 Dracula: Dead and Loving It (1995) Comedy|Horror 2.42 
13 Balto (1995) Adventure|Animation|Children 3.13 
14 Nixon (1995) Drama 3.83 politics president
15 Cutthroat Island (1995) Action|Adventure|Romance 3.0 
16 Casino (1995) Crime|Drama 3.93 Mafia
17 Sense and Sensibility (1995) Drama|Romance 3.78 Jane Austen
18 Four Rooms (1995) Comedy 3.7 
19 Ace

In [31]:
results = collection.query(
    query_texts=["Recommend me movies where Leonardo Dicaprio is the actor"],
    n_results=2
)


In [32]:
results

{'ids': [['79132', '99114']],
 'distances': [[0.9188426733016968, 1.0971240997314453]],
 'metadatas': [[{'title': 'Inception (2010)'},
   {'title': 'Django Unchained (2012)'}]],
 'embeddings': None,
 'documents': [['Inception (2010) Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX 4.07 Leonardo DiCaprio action alternate reality sci-fi thought-provoking visually appealing dreamlike surreal big budget clever complicated dead wife great soundtrack heist intellectual mindfuck philosophy psychological psychology suspense cerebral',
   'Django Unchained (2012) Action|Drama|Western 3.94 Christoph Waltz Great performances Humour Leonardo DiCaprio Quentin Tarantino Samuel L. Jackson Soundtrack action funny good soundtrack western']]}